In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import math
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    r2_score,
    mean_squared_log_error,
)
from tensorflow import keras
import datetime
from  sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from statsmodels.tsa.seasonal import seasonal_decompose
# holt winters 
# single exponential smoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing   
# double and triple exponential smoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing

label_encoder = LabelEncoder()

regressor = keras.models.Sequential()

LSTM = keras.layers.LSTM
Dropout = keras.layers.Dropout
Dense = keras.layers.Dense

ModuleNotFoundError: No module named 'tensorflow'

In [15]:
read_csv = pd.read_csv('/Users/victoroliveira/Desktop/vonix-py-statistic/src/vonixstatisc/data/pandas_data/data_fluency_queue_no_auto_dialer.csv')

f = read_csv.loc[(read_csv['handling_time'] > 10)]
df_= df.dropna(axis=0)
df_queue = df_.loc[df_['queue_id'] == 'equipez4']

In [19]:
#df_queue_agent = df_queue.loc[df_['agent_id'] == 1629]

In [20]:
#df_queue_agent_drop = df_queue_agent.drop('trunking_id', 1)
#df_queue_agent_drop2_ =df_queue_agent_drop.drop('call_type_id', 1)
#pd.set_option('display.max_rows', None)

In [21]:
df_queue

,agent_id,queue_id,direction,locality_id,call_type_id,hold_secs,ring_secs,initial_position,trunking_id,carrier_id,week_day,day,month,hour,minute,timestamp,handling_time
0,1000,equipez4,OUT,66000.0,3,0,7,0,1,1.0,4,23,February,20,2,1677185676,78
4,1000,equipez4,AUTO,31.0,5,2,0,1,1,3.0,4,23,February,21,2,1677186824,23
5,1116,equipez4,OUT,14.0,5,0,10,0,1,5.0,5,24,February,13,2,1677243851,48
6,1116,equipez4,OUT,11.0,5,0,50,0,1,3.0,5,24,February,13,2,1677244092,53
8,1116,equipez4,OUT,11.0,5,0,7,0,1,3.0,5,24,February,14,2,1677249481,597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96167,1629,equipez4,AUTO,81.0,5,10,1,1,1,1.0,3,12,April,23,4,1681342949,15
98825,1648,equipez4,OUT,21.0,5,0,7,0,1,5.0,4,13,April,20,4,1681419065,186
99568,1587,equipez4,OUT,71.0,5,0,8,0,1,3.0,4,13,April,21,4,1681422858,36
100175,1587,equipez4,OUT,71.0,5,0,6,0,1,3.0,5,14,April,14,4,1681483967,17


In [22]:
df_pre = df_queue.drop(['queue_id', 'timestamp', 'month'],1)
df_pre['agent_id']=  label_encoder.fit_transform(df_queue['agent_id'])
#df_pre['month']=  label_encoder.fit_transform(df_queue['month'])
df_pre['direction']=  label_encoder.fit_transform(df_queue['direction'])
df_pre['direction']=  label_encoder.fit_transform(df_queue['direction'])

/var/folders/3k/58r5jr214yn3hmkfb9pxlmjr0000gn/T/ipykernel_70793/1271444589.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_pre = df_queue.drop(['queue_id', 'timestamp', 'month'],1)


In [23]:
df_pre_r = df_pre.astype(int)

In [50]:
df_pre_r

,agent_id,direction,locality_id,call_type_id,hold_secs,ring_secs,initial_position,trunking_id,carrier_id,week_day,day,hour,minute,handling_time
0,0,1,66000,3,0,7,0,1,1,4,23,20,2,78
4,0,0,31,5,2,0,1,1,3,4,23,21,2,23
5,2,1,14,5,0,10,0,1,5,5,24,13,2,48
6,2,1,11,5,0,50,0,1,3,5,24,13,2,53
8,2,1,11,5,0,7,0,1,3,5,24,14,2,597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96167,52,0,81,5,10,1,1,1,1,3,12,23,4,15
98825,69,1,21,5,0,7,0,1,5,4,13,20,4,186
99568,34,1,71,5,0,8,0,1,3,4,13,21,4,36
100175,34,1,71,5,0,6,0,1,3,5,14,14,4,17


In [49]:
previsores = []
handling_time = []

previsores.append(df_pre_r.iloc[:,0:13])

previsores

#handling_time =df_pre_r['handling_time']


[        agent_id  direction  locality_id  call_type_id  hold_secs  ring_secs  \
 0              0          1        66000             3          0          7   
 4              0          0           31             5          2          0   
 5              2          1           14             5          0         10   
 6              2          1           11             5          0         50   
 8              2          1           11             5          0          7   
 ...          ...        ...          ...           ...        ...        ...   
 96167         52          0           81             5         10          1   
 98825         69          1           21             5          0          7   
 99568         34          1           71             5          0          8   
 100175        34          1           71             5          0          6   
 100189        34          1           34             5          0         20   
 
         initial_position 

In [ ]:
previsores = df_pre_r

In [32]:
regressor = keras.models.Sequential()

LSTM = keras.layers.LSTM
Dropout = keras.layers.Dropout
Dense = keras.layers.Dense


In [47]:
regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores.shape[1], 13)))
regressor.add(Dropout(0.3))     


regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (previsores.shape[1], 13)))
regressor.add(Dropout(0.3))    

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (previsores.shape[1], 13)))
regressor.add(Dropout(0.3))  

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (previsores.shape[1], 13)))
regressor.add(Dropout(0.3)) 


regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3)) 

regressor.add(Dense(units = 1, activation = 'sigmoid'))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])

